# 对得到的候选数据集Admin_id和Item_id进行清洗并提取特征信息以放入模型中进行推理

## 1. 读取🐟佬提交数据生成特征，之后使用网络进行重新排序

使用测试集生成item特征信息，主要体现在num_sell上面

In [3]:
import pandas as pd
import numpy as np
import pickle
import os

In [156]:
yulao_sub = pd.read_csv('../submission.csv',header=None)

In [158]:
yulao_sub.shape

(11398, 31)

In [340]:
# 将🐟佬的提交信息提取出来并构成dataframe格式，将用户信息扩充到30个维度
item_id_name = yulao_sub.iloc[:,1:].values.flatten()
admin_id_name = np.repeat(yulao_sub[0].values,30)
yulao_sub_feature = pd.DataFrame(columns=['buyer_admin_id','item_id'])
yulao_sub_feature['buyer_admin_id'] = admin_id_name 
yulao_sub_feature['item_id'] = item_id_name

In [341]:
yulao_sub_feature[:2]

,buyer_admin_id,item_id
0,152,8410857
1,152,7937154


### Merge客户信息

处理鱼佬admin_id 和item_id数据

+ 首先查看客户信息，客户信息可能存在缺失情况

使用测试集中的数据作为商品特征

In [342]:
# 之前生成的测试集上用户的特征信息，根据用户在测试集上的购买记录生成得到，已经去除了用户购买商品不在商品表中的数据
with open('features/test_admin_features.pkl','rb') as f:
    test_admin_features = pickle.load(f)  

In [343]:
# 合并信息
yulao_sub_feature = pd.merge(yulao_sub_feature, test_admin_features, on='buyer_admin_id',how='outer')
# 缺失用户ID信息
print('缺失用户id为： %s'%yulao_sub_feature[yulao_sub_feature.isna().values==True]['buyer_admin_id'].unique())

缺失用户id为： [ 262400  495938  713555 1913825 2101357]


In [354]:
yulao_sub_feature.shape

(341940, 15)

In [347]:
yulao_sub_feature[:4]  # 合并用户信息之后的数据

,buyer_admin_id,item_id,num_shop,num_item,pop_unpop_rate,item_price_sum,item_price_max,item_price_min,item_price_mean,item_price_median,num_item_cate,num_store_cate,shoptime_1,shoptime_2,shoptime_3
0,152,8410857,7.0,7.0,0.0,14538.0,7479.0,200.0,2076.857143,1279.0,4.0,7.0,7.0,0.0,0.0
1,152,7937154,7.0,7.0,0.0,14538.0,7479.0,200.0,2076.857143,1279.0,4.0,7.0,7.0,0.0,0.0
2,152,8472223,7.0,7.0,0.0,14538.0,7479.0,200.0,2076.857143,1279.0,4.0,7.0,7.0,0.0,0.0
3,152,4016066,7.0,7.0,0.0,14538.0,7479.0,200.0,2076.857143,1279.0,4.0,7.0,7.0,0.0,0.0


In [349]:
# 统计缺失值情况
yulao_sub_feature.isna().sum()

buyer_admin_id         0
item_id                0
num_shop             150
num_item             150
pop_unpop_rate       150
item_price_sum       150
item_price_max       150
item_price_min       150
item_price_mean      150
item_price_median    150
num_item_cate        150
num_store_cate       150
shoptime_1           150
shoptime_2           150
shoptime_3           150
dtype: int64

 读取原始test的csv文件，查看缺失用户的购买记录

In [344]:
test_df_src = pd.read_csv('../csv/Antai_AE_round1_test_20190626.csv') 

In [345]:
test_df_src[test_df_src['buyer_admin_id'] == 262400]

,buyer_country_id,buyer_admin_id,item_id,create_order_time,irank
88847,yy,262400,10045561,2018-08-21 12:36:04,5
88848,yy,262400,10045561,2018-08-21 12:36:04,3
88849,yy,262400,10045561,2018-08-21 12:36:04,6
88850,yy,262400,10045561,2018-08-21 12:36:04,8
88851,yy,262400,10045561,2018-08-21 12:36:04,13
88852,yy,262400,10045561,2018-08-21 12:36:04,9
88853,yy,262400,10045561,2018-08-21 12:36:04,4
88854,yy,262400,10045561,2018-08-21 12:36:04,2
88855,yy,262400,10045561,2018-08-21 12:36:04,11
88856,yy,262400,10045561,2018-08-21 12:36:04,12


In [346]:
item_df[item_df['item_id'] ==10045561]

,item_id,cate_id,store_id,item_price


+ 小结：

有五个顾客id为[262400,  495938,  713555, 1913825, 2101357]他们从头至尾只买了一件商品，且该商品没有出现在商品表中，所以可以直接推断这5个客户之后还是只买该商品

这5个用户不用放入模型进行训练，所以直接删除缺失的admin_id信息

In [355]:
yulao_sub_feature.dropna(axis=0, how='any', inplace=True)  # 删除缺失值所在的行

In [358]:
yulao_sub_feature.isna().sum()

buyer_admin_id       0
item_id              0
num_shop             0
num_item             0
pop_unpop_rate       0
item_price_sum       0
item_price_max       0
item_price_min       0
item_price_mean      0
item_price_median    0
num_item_cate        0
num_store_cate       0
shoptime_1           0
shoptime_2           0
shoptime_3           0
dtype: int64

### Merge商品信息

In [351]:
# item 可能同时存在训练集和测试集中，所以同时读取两个数据集中item特征信息进行统计
with open('features/test_item_features.pkl','rb') as f:
    test_item_features = pickle.load(f)

In [265]:
test_item_features[:3]

,item_id,num_sell,cate_id,store_id,item_price
0,189045,1,1506,11599,237
1,10713670,1,1911,39939,101
2,10546322,1,1553,65849,40


In [359]:
yulao_sub_feature_testitem = pd.merge(yulao_sub_feature,test_item_features,on='item_id',how='left') # merge训练集中商品信息
yulao_sub_feature_testitem.isna().sum() # 统计缺失值

buyer_admin_id         0
item_id                0
num_shop               0
num_item               0
pop_unpop_rate         0
item_price_sum         0
item_price_max         0
item_price_min         0
item_price_mean        0
item_price_median      0
num_item_cate          0
num_store_cate         0
shoptime_1             0
shoptime_2             0
shoptime_3             0
num_sell             412
cate_id              412
store_id             412
item_price           412
dtype: int64

In [362]:
missing_item  = yulao_sub_feature_testitem[yulao_sub_feature_testitem.isna().values==True]['item_id'].unique()

In [369]:
print('缺失商品的个数有： %s'%missing_item.shape[0])

缺失商品的个数有： 298


In [367]:
set(item_df['item_id'].to_list()).intersection(set(missing_item))  # 所以缺失的这些商品信息就是在商品属性表中缺失的商品信息

set()

如果直接将这些缺失删除是否会影响admin的个数？

In [394]:
yulao_sub_feature_testitem_drop = yulao_sub_feature_testitem.dropna(axis=0,how='any')

查看缺失用户情况

In [377]:
set(yulao_sub_feature_testitem_drop['buyer_admin_id'].unique()).difference(set(yulao_sub_feature_testitem['buyer_admin_id'].unique()))

set()

所以用户没有减少，只是减少了一些item，所以这里就将其进行删除，该商品不放入模型进行训练，在进行提交的时候这些商品就保持原来所在的位置

In [395]:
yulao_sub_feature_testitem_drop[:4]

,buyer_admin_id,item_id,num_shop,num_item,pop_unpop_rate,item_price_sum,item_price_max,item_price_min,item_price_mean,item_price_median,num_item_cate,num_store_cate,shoptime_1,shoptime_2,shoptime_3,num_sell,cate_id,store_id,item_price
0,152,8410857,7.0,7.0,0.0,14538.0,7479.0,200.0,2076.857143,1279.0,4.0,7.0,7.0,0.0,0.0,1.0,2686.0,6213.0,800.0
1,152,7937154,7.0,7.0,0.0,14538.0,7479.0,200.0,2076.857143,1279.0,4.0,7.0,7.0,0.0,0.0,1.0,2686.0,40393.0,1301.0
2,152,8472223,7.0,7.0,0.0,14538.0,7479.0,200.0,2076.857143,1279.0,4.0,7.0,7.0,0.0,0.0,1.0,2686.0,79168.0,1234.0
3,152,4016066,7.0,7.0,0.0,14538.0,7479.0,200.0,2076.857143,1279.0,4.0,7.0,7.0,0.0,0.0,1.0,3834.0,54345.0,2245.0


In [396]:
yulao_sub_feature_testitem_drop['buyer_admin_id'].unique().shape

(11393,)

修改item数据类型并生成sub pickle文件

In [397]:
yulao_sub_feature_testitem_drop['item_price'] = yulao_sub_feature_testitem_drop['item_price'].astype(np.int32)
yulao_sub_feature_testitem_drop['num_sell'] = yulao_sub_feature_testitem_drop['num_sell'].astype(np.int32)
yulao_sub_feature_testitem_drop['cate_id'] = yulao_sub_feature_testitem_drop['cate_id'].astype(np.int32)
yulao_sub_feature_testitem_drop['store_id'] = yulao_sub_feature_testitem_drop['store_id'].astype(np.int32)

/home/ai/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/ai/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ai/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

In [398]:
yulao_sub_feature_testitem_drop[:3]

,buyer_admin_id,item_id,num_shop,num_item,pop_unpop_rate,item_price_sum,item_price_max,item_price_min,item_price_mean,item_price_median,num_item_cate,num_store_cate,shoptime_1,shoptime_2,shoptime_3,num_sell,cate_id,store_id,item_price
0,152,8410857,7.0,7.0,0.0,14538.0,7479.0,200.0,2076.857143,1279.0,4.0,7.0,7.0,0.0,0.0,1,2686,6213,800
1,152,7937154,7.0,7.0,0.0,14538.0,7479.0,200.0,2076.857143,1279.0,4.0,7.0,7.0,0.0,0.0,1,2686,40393,1301
2,152,8472223,7.0,7.0,0.0,14538.0,7479.0,200.0,2076.857143,1279.0,4.0,7.0,7.0,0.0,0.0,1,2686,79168,1234


In [399]:
yulao_sub_feature_testitem_drop['num_sell'].max()

1359

In [403]:
yulao_sub_feature_testitem_drop['label'] = -1

/home/ai/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [404]:
yulao_sub_feature_testitem_drop.isna().sum()

buyer_admin_id       0
item_id              0
num_shop             0
num_item             0
pop_unpop_rate       0
item_price_sum       0
item_price_max       0
item_price_min       0
item_price_mean      0
item_price_median    0
num_item_cate        0
num_store_cate       0
shoptime_1           0
shoptime_2           0
shoptime_3           0
num_sell             0
cate_id              0
store_id             0
item_price           0
label                0
dtype: int64

In [406]:
with open('model_data/yulao_sub_data1.pkl', 'wb') as output_file: # 保存pickle文件
        pickle.dump(yulao_sub_feature_testitem_drop, output_file)

## 使用训练集数据中购买记录生成item特征，主要体现在num_sell上

In [4]:
yulao_sub = pd.read_csv('../submission.csv',header=None)
yulao_sub.shape

(11398, 31)

In [17]:
# 将🐟佬的提交信息提取出来并构成dataframe格式，将用户信息扩充到30个维度
item_id_name = yulao_sub.iloc[:,1:].values.flatten()
admin_id_name = np.repeat(yulao_sub[0].values,30)
yulao_sub_feature = pd.DataFrame(columns=['buyer_admin_id','item_id'])
yulao_sub_feature['buyer_admin_id'] = admin_id_name 
yulao_sub_feature['item_id'] = item_id_name
yulao_sub_feature[:3]

,buyer_admin_id,item_id
0,152,8410857
1,152,7937154
2,152,8472223


+ Merge客户信息

和前面的操作一样，用户信息肯定是从test集中提取得到

In [15]:
# 之前生成的测试集上用户的特征信息，根据用户在测试集上的购买记录生成得到，已经去除了用户购买商品不在商品表中的数据
with open('features/test_admin_features.pkl','rb') as f:
    test_admin_features = pickle.load(f)  

In [18]:
# 合并信息
yulao_sub_feature = pd.merge(yulao_sub_feature, test_admin_features, on='buyer_admin_id',how='outer')
# 缺失用户ID信息
print('缺失用户id为： %s'%yulao_sub_feature[yulao_sub_feature.isna().values==True]['buyer_admin_id'].unique())

缺失用户id为： [ 262400  495938  713555 1913825 2101357]


In [21]:
yulao_sub_feature.dropna(axis=0, how='any', inplace=True)  # 删除缺失值所在的行

+ merge商品信息

商品信息中主要的特征为null_sell, 如果训练集中有num_sell就用训练集中的数据，没有的话就用测试集中的数据

In [23]:
with open('features/test_item_features.pkl','rb') as f:
    test_item_features = pickle.load(f)
with open('features/train_item_features.pkl','rb') as f:
    train_item_features = pickle.load(f)

In [87]:
yulao_sub_feature_trainitem = pd.merge(yulao_sub_feature,train_item_features,on='item_id',how='left') # merge训练集中商品信息
yulao_sub_feature_trainitem.isna().sum() # 统计缺失值

buyer_admin_id           0
item_id                  0
num_shop                 0
num_item                 0
pop_unpop_rate           0
item_price_sum           0
item_price_max           0
item_price_min           0
item_price_mean          0
item_price_median        0
num_item_cate            0
num_store_cate           0
shoptime_1               0
shoptime_2               0
shoptime_3               0
num_sell             27200
cate_id              27200
store_id             27200
item_price           27200
dtype: int64

In [88]:
yulao_sub_feature_testitem = pd.merge(yulao_sub_feature,test_item_features,on='item_id',how='left') # merge训练集中商品信息
yulao_sub_feature_testitem.isna().sum() # 统计缺失值

buyer_admin_id         0
item_id                0
num_shop               0
num_item               0
pop_unpop_rate         0
item_price_sum         0
item_price_max         0
item_price_min         0
item_price_mean        0
item_price_median      0
num_item_cate          0
num_store_cate         0
shoptime_1             0
shoptime_2             0
shoptime_3             0
num_sell             412
cate_id              412
store_id             412
item_price           412
dtype: int64

In [95]:
yulao_sub_feature_item = yulao_sub_feature_trainitem.fillna(yulao_sub_feature_testitem)  # 用test_item中的数据填充缺失值

In [96]:
yulao_sub_feature_item[:3]

,buyer_admin_id,item_id,num_shop,num_item,pop_unpop_rate,item_price_sum,item_price_max,item_price_min,item_price_mean,item_price_median,num_item_cate,num_store_cate,shoptime_1,shoptime_2,shoptime_3,num_sell,cate_id,store_id,item_price
0,152,8410857,7.0,7.0,0.0,14538.0,7479.0,200.0,2076.857143,1279.0,4.0,7.0,7.0,0.0,0.0,2.0,2686.0,6213.0,800.0
1,152,7937154,7.0,7.0,0.0,14538.0,7479.0,200.0,2076.857143,1279.0,4.0,7.0,7.0,0.0,0.0,61.0,2686.0,40393.0,1301.0
2,152,8472223,7.0,7.0,0.0,14538.0,7479.0,200.0,2076.857143,1279.0,4.0,7.0,7.0,0.0,0.0,6.0,2686.0,79168.0,1234.0


删除缺失值，准备输出特征

In [98]:
yulao_sub_feature_item = yulao_sub_feature_item.dropna(axis=0,how='any')
yulao_sub_feature_item['item_price'] = yulao_sub_feature_item['item_price'].astype(np.int32)
yulao_sub_feature_item['num_sell'] = yulao_sub_feature_item['num_sell'].astype(np.int32)
yulao_sub_feature_item['cate_id'] = yulao_sub_feature_item['cate_id'].astype(np.int32)
yulao_sub_feature_item['store_id'] = yulao_sub_feature_item['store_id'].astype(np.int32)

In [99]:
yulao_sub_feature_item.isna().sum()

buyer_admin_id       0
item_id              0
num_shop             0
num_item             0
pop_unpop_rate       0
item_price_sum       0
item_price_max       0
item_price_min       0
item_price_mean      0
item_price_median    0
num_item_cate        0
num_store_cate       0
shoptime_1           0
shoptime_2           0
shoptime_3           0
num_sell             0
cate_id              0
store_id             0
item_price           0
dtype: int64

In [101]:
yulao_sub_feature_item['label'] = -1

In [102]:
with open('model_data/yulao_sub_data2.pkl', 'wb') as output_file: # 保存pickle文件
        pickle.dump(yulao_sub_feature_item, output_file)